# QuASAr Partition Inspector

Use this notebook to generate a benchmark circuit from the unified benchmark suite and inspect how the QuASAr planner partitions it under different configuration options.

Set the parameter dictionaries below to match the circuit family and planner overrides that you would like to investigate. The helper functions defined in this notebook summarise the resulting plan so you can confirm that the partitioning behaviour matches expectations for large benchmark circuits.

In [ ]:
from __future__ import annotations

from typing import Any, Dict, Iterable, List, Sequence

try:
    import pandas as pd  # type: ignore[import]
except Exception:
    pd = None

from IPython.display import Markdown, display

from benchmarks.unified import generate_benchmark_circuit
from quasar.analyzer import analyze
from quasar.planner import PlannerConfig, plan as build_plan


In [ ]:
# --- Circuit and planner configuration ---------------------------------------------------
circuit_options: Dict[str, Any] = {
    'num_qubits': 16,
    'depth': 48,
    'family': 'mixed',
    'block_size': 8,
    'cutoff': 0.75,
    'tail_type': 'sparse',
    'seed': 7,
}

planner_overrides: Dict[str, Any] = {
    'max_ram_gb': 4.0,
    'prefer_dd': True,
    'hybrid_clifford_tail': True,
    'max_candidate_plans': 4,
}


In [ ]:
def make_planner_config(overrides: Dict[str, Any]) -> PlannerConfig:
    cfg = PlannerConfig()
    for key, value in overrides.items():
        if hasattr(cfg, key):
            setattr(cfg, key, value)
        else:
            raise AttributeError(f"PlannerConfig has no field {key!r}")
    return cfg

def summarize_plan_nodes(plan) -> List[Dict[str, Any]]:
    rows: List[Dict[str, Any]] = []
    for seq, node in enumerate(plan.qusds):
        metrics = dict(getattr(node, 'metrics', {}) or {})
        meta = dict(getattr(node, 'meta', {}) or {})
        rows.append({
            'seq_index': seq,
            'qusd_id': node.id,
            'backend': node.backend,
            'qubits': list(node.qubits),
            'num_qubits': metrics.get('num_qubits'),
            'depth': metrics.get('depth'),
            'two_qubit_gates': metrics.get('two_qubit_gates'),
            't_count': metrics.get('t_count'),
            'sparsity': metrics.get('sparsity'),
            'planner_reason': meta.get('planner_reason'),
            'chain_id': meta.get('chain_id'),
            'successors': [succ.id for succ in getattr(node, 'successors', [])],
        })
    return rows

def display_table(rows: Sequence[Dict[str, Any]]) -> None:
    if not rows:
        display(Markdown('No plan nodes to display.'))
        return
    if pd is not None:
        display(pd.DataFrame(rows))
        return
    headers = list(rows[0].keys())
    lines = [
        '| ' + ' | '.join(headers) + ' |',
        '| ' + ' | '.join(['---'] * len(headers)) + ' |',
    ]
    for row in rows:
        values = [str(row.get(h, '')) for h in headers]
        lines.append('| ' + ' | '.join(values) + ' |')
    display(Markdown('
'.join(lines)))

def inspect_partition(circuit_args: Dict[str, Any], overrides: Dict[str, Any]) -> Dict[str, Any]:
    circuit = generate_benchmark_circuit(**circuit_args)
    analysis = analyze(circuit)
    cfg = make_planner_config(overrides)
    planned = build_plan(analysis.plan, cfg)

    display(Markdown('### Global circuit metrics'))
    display_table([{k: analysis.metrics_global[k] for k in sorted(analysis.metrics_global)}])

    display(Markdown('### Planned partition'))
    display_table(summarize_plan_nodes(planned))

    display(Markdown('### Planner metadata'))
    planner_meta = dict(getattr(planned, 'meta', {}))
    planner_meta.update({'estimated_cost': getattr(planned, 'estimated_cost', None)})
    display_table([{k: planner_meta[k] for k in sorted(planner_meta)}])

    display(Markdown('### Decision trace'))
    trace = getattr(planned, 'decision_trace', [])
    if trace:
        trace_rows = [{'step': idx, 'decision': entry} for idx, entry in enumerate(trace)]
    else:
        trace_rows = [{'step': 0, 'decision': '(none)'}]
    display_table(trace_rows)

    return {
        'circuit': circuit,
        'analysis': analysis,
        'plan': planned,
        'planner_config': cfg,
    }


In [ ]:
# Run the inspection with the current configuration
results = inspect_partition(circuit_options, planner_overrides)
